# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition.

# 1) Importing data

In [1]:
import numpy as np
import pandas as pd

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [9]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = 'subtask_4b/subtask4b_collection_data.pkl' #MODIFY PATH

In [10]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [11]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [12]:
df_collection.head(3)

cord_uid source_x                                              title  \
162  umvrwgaw      PMC  Professional and Home-Made Face Masks Reduce E...   
611  spiud6ok      PMC                               The Failure of R (0)   
918  aclzp3iy      PMC  Pulmonary sequelae in a patient recovered from...   

                              doi       pmcid pubmed_id      license  \
162  10.1371/journal.pone.0002618  PMC2440799  18612429        cc-by   
611           10.1155/2011/527610  PMC3157160  21860658        cc-by   
918       10.4103/0970-2113.99118  PMC3424870  22919170  cc-by-nc-sa   

                                              abstract publish_time  \
162  BACKGROUND: Governments are preparing for a po...   2008-07-09   
611  The basic reproductive ratio, R (0), is one of...   2011-08-16   
918  The pandemic of swine flu (H1N1) influenza spr...         2012   

                                               authors  \
162  van der Sande, Marianne; Teunis, Peter; Sabel,...   
611      Li, Jing; Blakeley, Daniel; Smith?, Robert J.   
918  Singh, Virendra; Sharma, Bharat Bhushan; Patel...   

                     journal  mag_id who_covidence_id arxiv_id     label  \
162                 PLoS One     NaN              NaN      NaN  umvrwgaw   
611  Comput Math Methods Med     NaN              NaN      NaN  spiud6ok   
918               Lung India     NaN              NaN      NaN  aclzp3iy   

          time       timet  
162 2008-07-09  1215561600  
611 2011-08-16  1313452800  
918 2012-01-01  1325376000

## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [13]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_QUERY_TRAIN_DATA = 'subtask_4b/subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = 'subtask_4b/subtask4b_query_tweets_dev.tsv' #MODIFY PATH

In [14]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [15]:
df_query_train.head()

post_id                                         tweet_text  cord_uid
0        0  Oral care in rehabilitation medicine: oral vul...  htlvpvz5
1        1  this study isn't receiving sufficient attentio...  4kfl29ul
2        2  thanks, xi jinping. a reminder that this study...  jtwb17u8
3        3  Taiwan - a population of 23 million has had ju...  0w9k8iy1
4        4  Obtaining a diagnosis of autism in lower incom...  tiqksd69

In [16]:
df_query_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     12853 non-null  int64 
 1   tweet_text  12853 non-null  object
 2   cord_uid    12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [17]:
df_query_dev.head()

post_id                                         tweet_text  cord_uid
0       16  covid recovery: this study from the usa reveal...  3qvh482o
1       69  "Among 139 clients exposed to two symptomatic ...  r58aohnu
2       73  I recall early on reading that researchers who...  sts48u9i
3       93  You know you're credible when NIH website has ...  3sr2exq9
4       96  Resistance to antifungal medications is a grow...  ybwwmyqy

In [18]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


# 2) Running the baseline
The following code runs a BM25 baseline.


In [19]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi
from tqdm import tqdm

In [20]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [21]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:5]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [22]:
# Retrieve topk candidates using the BM25 model
df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))

## 2a) Improvement suggestions in baseline

### 1. Text Preprocessing
The baseline has basic tockenization, we can also add the following to it:
- Lowercasing
- stepword removal
- stemming
all this using 'nltk' package

In [31]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TreebankWordTokenizer
# from nltk.tokenize import word_tokenize #do not work well

tokenizer = TreebankWordTokenizer()
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    if pd.isna(text):
        return []
    tokens = tokenizer.tokenize(text.lower())
    filtered = [stemmer.stem(w) for w in tokens if w.isalnum() and w not in stop_words]
    return filtered

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kartikarya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kartikarya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2. Adding Inverted Index
Explicitly adding Inverted index to store Document IDs, Term Frequencies, Document Lengths, Average Document Length. This will allow us to calculate TF-IDF alomg with BM25 scoring \\

- then adding a function to include TF-IDF scoring model


In [32]:
from collections import defaultdict

inverted_index = defaultdict(list)
doc_lengths = {}
total_length = 0

for idx, row in df_collection.iterrows():
    doc_id = row['cord_uid']
    text = str(row['title']) + ' ' + str(row['abstract'])
    tokens = preprocess_text(text)
    doc_lengths[doc_id] = len(tokens)
    total_length += len(tokens)
    term_freq = defaultdict(int)
    for token in tokens:
        term_freq[token] += 1
    for token, freq in term_freq.items():
        inverted_index[token].append((doc_id, freq))

avg_doc_length = total_length / len(doc_lengths)
print(f"Indexed {len(doc_lengths)} documents with average length {avg_doc_length:.2f}")

import math

def compute_tfidf_scores(query, index, doc_lengths, total_docs):
    tokens = preprocess_text(query)
    scores = defaultdict(float)
    for token in tokens:
        postings = index.get(token, [])
        df = len(postings)
        if df == 0:
            continue
        idf = math.log(total_docs / df)
        for doc_id, tf in postings:
            scores[doc_id] += math.log(1 + tf) * idf
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

Indexed 7718 documents with average length 139.72


# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [ ]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

# tfidf_topk_train = []
# for query in df_query_train["tweet_text"]:
#     ranked_docs = compute_tfidf_scores(query, inverted_index, doc_lengths, len(doc_lengths))[:10]
#     doc_ids = [doc for doc, _ in ranked_docs]
#     tfidf_topk_train.append(doc_ids)

# df_query_train["tfidf_topk"] = tfidf_topk_train

# using 'lambda function' uisng chatGPT coz simple tf-idf was giving error :}
df_query_train["tfidf_topk"] = df_query_train["tweet_text"].apply(
    lambda q: [doc for doc, _ in compute_tfidf_scores(q, inverted_index, doc_lengths, len(doc_lengths))[:10]]
)

df_query_dev["tfidf_topk"] = df_query_dev["tweet_text"].apply(
    lambda q: [doc for doc, _ in compute_tfidf_scores(q, inverted_index, doc_lengths, len(doc_lengths))[:10]]
)



In [37]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')

# Printed MRR@k results in the following format: {k: MRR@k}
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")
print('\n')

# adding TF-IDF results here
results_train_tfidf = get_performance_mrr(df_query_train, 'cord_uid', 'tfidf_topk')
results_dev_tfidf = get_performance_mrr(df_query_dev, 'cord_uid', 'tfidf_topk')

print(f"TF-IDF Results on the train set: {results_train_tfidf}")
print(f"TF-IDF Results on the dev set: {results_dev_tfidf}")

Results on the train set: {1: np.float64(0.5079747918773827), 5: np.float64(0.5508999196037242), 10: np.float64(0.5508999196037242)}
Results on the dev set: {1: np.float64(0.505), 5: np.float64(0.5520357142857142), 10: np.float64(0.5520357142857142)}


TF-IDF Results on the train set: {1: np.float64(0.45335719287325915), 5: np.float64(0.5150950491454653), 10: np.float64(0.5226302858081431)}
TF-IDF Results on the dev set: {1: np.float64(0.45785714285714285), 5: np.float64(0.5198571428571428), 10: np.float64(0.5266734693877552)}


# 4) Exporting results to prepare the submission on Codalab

In [21]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [22]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')